In [19]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine, text
import os
import glob
from pathlib import Path
import re
import unicodedata


In [20]:
def conectar_postgres():
    """Conecta ao PostgreSQL no Docker"""
    try:
        host = 'localhost'
        port = '5432'
        database = 'postgres'
        username = 'postgres'
        password = 'minhasenha'
        
        connection_string = f'postgresql://{username}:{password}@{host}:{port}/{database}'
        engine = create_engine(connection_string)
        print("✅ Conexão com PostgreSQL estabelecida!")
        return engine
    
    except Exception as e:
        print(f"❌ Erro ao conectar: {e}")
        return None
engine = conectar_postgres()

✅ Conexão com PostgreSQL estabelecida!


In [ ]:
query_zmb51 = """
-- zmb51: movimentações no último ano
SELECT
    material,
    centro,
    qtd_um_registro,
    canal,
    data_de_lancamento
FROM power_bi.zmb51
WHERE material IN (
    'A8K3430',
    'A8K3230',
    'AAV8230',
    'AAV8330',
    'T671600'
)
AND data_de_lancamento >= CURRENT_DATE - INTERVAL '12 months';


"""
query_zstok = """
-- zstok: estoque atual por material
SELECT
    material,
    estoque_total,
    cen AS centro
FROM power_bi.zstok
WHERE material IN (
    'A8K3430',
    'A8K3230',
    'AAV8230',
    'AAV8330',
    'T671600'
);
"""
query_fup = """
-- fup: previsões de entrada por material
SELECT
    material,
    qtde_pedido,
    data_prev_entrada,
    data_de_remessa
FROM power_bi.fup
WHERE material IN (
    'A8K3430',
    'A8K3230',
    'AAV8230',
    'AAV8330',
    'T671600'
);
"""

In [22]:
zmb51 = pd.read_sql_query(query_zmb51, engine)
zstok = pd.read_sql_query(query_zstok, engine)
fup = pd.read_sql_query(query_fup, engine)

In [30]:
zmb51.columns

Index(['material', 'centro', 'qtd_um_registro', 'canal', 'data_de_lancamento'], dtype='object')

In [27]:
zstok_pivot = zstok.pivot_table(index='material', columns='centro', values='estoque_total')

In [28]:
zstok_pivot

centro,CE05,CE07,CE09,CE10,CE11,CE13
material,,,,,,
A8K3230,27.0,16.0,15.0,NaN,12.0,485.0
A8K3430,118.0,9.0,7.0,1.0,36.0,232.0
AAV8230,198.0,21.5,77.0,NaN,164.0,1980.0
AAV8330,NaN,35.0,68.0,NaN,144.0,2088.0


In [ ]:
zmb51['data_de_lancamento'] = pd.to_datetime(zmb51['data_de_lancamento'])

zmb51['ano_mes'] = zmb51['data_de_lancamento'].dt.to_period('M')

In [35]:
zmb51_sem_ce07 = zmb51[zmb51['centro'] != 'CE07']

zmb51_ce07 = zmb51[zmb51['centro'] == 'CE07']


In [36]:
zmb51_pivot_sem_ce07 = zmb51_sem_ce07.pivot_table(
    index='material',
    columns='ano_mes',
    values='qtd_um_registro',
    aggfunc='sum'
)

zmb51_pivot_ce07 = zmb51_ce07.pivot_table(
    index='material',
    columns='ano_mes',
    values='qtd_um_registro',
    aggfunc='sum'
    )

In [37]:
zmb51_pivot_sem_ce07

ano_mes,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,2025-04,2025-05
material,,,,,,,,,,,,
A8K3230,99.0,152.0,62.0,55.0,103.0,66.0,83.0,5.0,74.0,64.0,52.0,56.0
A8K3430,91.0,192.0,65.0,74.0,90.0,121.0,142.0,10.0,156.0,138.0,46.0,96.0
AAV8230,161.0,938.0,436.0,543.0,445.0,522.0,1392.0,388.0,582.0,751.0,565.0,456.0
AAV8330,391.0,394.0,419.0,484.0,475.0,454.0,1391.0,350.0,568.0,635.0,482.0,416.0
T671600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN


In [38]:
zmb51_pivot_ce07

ano_mes,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,2025-04,2025-05
material,,,,,,,,,,,,
A8K3230,6.0,5.0,7.0,7.0,6.0,5.0,5.0,9.0,2.0,4.0,5.0,6.0
A8K3430,5.0,2.0,3.0,5.0,5.0,5.0,5.0,8.0,2.0,2.0,6.0,5.0
AAV8230,6.0,9.0,14.0,17.0,19.0,12.0,23.0,7.0,19.0,20.0,23.0,23.0
AAV8330,10.0,12.0,11.0,15.0,19.0,14.0,23.0,8.0,16.0,15.0,21.0,28.0
T671600,1.0,NaN,NaN,1.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,1.0


In [ ]:
zstok_pivot_ce07 = zstok_pivot[['CE07']]  

In [42]:
zstok_pivot_ce07

centro,CE07
material,
A8K3230,16.0
A8K3430,9.0
AAV8230,21.5
AAV8330,35.0


In [43]:
zmb51_pivot_ce07

ano_mes,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,2025-04,2025-05
material,,,,,,,,,,,,
A8K3230,6.0,5.0,7.0,7.0,6.0,5.0,5.0,9.0,2.0,4.0,5.0,6.0
A8K3430,5.0,2.0,3.0,5.0,5.0,5.0,5.0,8.0,2.0,2.0,6.0,5.0
AAV8230,6.0,9.0,14.0,17.0,19.0,12.0,23.0,7.0,19.0,20.0,23.0,23.0
AAV8330,10.0,12.0,11.0,15.0,19.0,14.0,23.0,8.0,16.0,15.0,21.0,28.0
T671600,1.0,NaN,NaN,1.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,1.0


In [ ]:
zmb51_pivot_ce07 = zmb51_pivot_ce07.sort_index(axis=1)

zmb51_pivot_ce07['media_3m'] = zmb51_pivot_ce07.iloc[:, -3:].mean(axis=1)
zmb51_pivot_ce07['media_6m'] = zmb51_pivot_ce07.iloc[:, -6:].mean(axis=1)


In [47]:
zmb51_pivot_ce07

ano_mes,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,2025-04,2025-05,media_3m,media_6m
material,,,,,,,,,,,,,,
A8K3230,6.0,5.0,7.0,7.0,6.0,5.0,5.0,9.0,2.0,4.0,5.0,6.0,5.000000,5.166667
A8K3430,5.0,2.0,3.0,5.0,5.0,5.0,5.0,8.0,2.0,2.0,6.0,5.0,4.333333,4.555556
AAV8230,6.0,9.0,14.0,17.0,19.0,12.0,23.0,7.0,19.0,20.0,23.0,23.0,22.000000,19.000000
AAV8330,10.0,12.0,11.0,15.0,19.0,14.0,23.0,8.0,16.0,15.0,21.0,28.0,21.333333,18.222222
T671600,1.0,NaN,NaN,1.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,1.0,1.000000,1.000000


In [48]:
zmb51_estoque_ce07 = pd.concat([zmb51_pivot_ce07, zstok_pivot_ce07], axis=1)


In [49]:
zmb51_estoque_ce07

,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,2025-04,2025-05,media_3m,media_6m,CE07
material,,,,,,,,,,,,,,,
A8K3230,6.0,5.0,7.0,7.0,6.0,5.0,5.0,9.0,2.0,4.0,5.0,6.0,5.000000,5.166667,16.0
A8K3430,5.0,2.0,3.0,5.0,5.0,5.0,5.0,8.0,2.0,2.0,6.0,5.0,4.333333,4.555556,9.0
AAV8230,6.0,9.0,14.0,17.0,19.0,12.0,23.0,7.0,19.0,20.0,23.0,23.0,22.000000,19.000000,21.5
AAV8330,10.0,12.0,11.0,15.0,19.0,14.0,23.0,8.0,16.0,15.0,21.0,28.0,21.333333,18.222222,35.0
T671600,1.0,NaN,NaN,1.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,1.0,1.000000,1.000000,NaN


In [52]:
zmb51_estoque_ce07.reset_index().to_json(
    r"C:\Users\fsp_adolpho.salvador\Desktop\Konica Minolta\Desktop Cloud - Documentos\Desktop\Py\Analise\trasferencias\zmb51_estoque_ce07.json",
    orient="records",
    indent=4
)

In [72]:
zstok_pivot_sem_ce07 = zstok_pivot.drop(columns=['CE07'])

In [73]:
zstok_pivot_sem_ce07

centro,CE05,CE09,CE10,CE11,CE13
material,,,,,
A8K3230,27.0,15.0,NaN,12.0,485.0
A8K3430,118.0,7.0,1.0,36.0,232.0
AAV8230,198.0,77.0,NaN,164.0,1980.0
AAV8330,NaN,68.0,NaN,144.0,2088.0


In [74]:
zstok_pivot_sem_ce07['media_3m'] = zstok_pivot_sem_ce07.iloc[:, -3:].mean(axis=1)
zstok_pivot_sem_ce07['media_6m'] = zstok_pivot_sem_ce07.iloc[:, -6:].mean(axis=1)

In [75]:
zmb51_estoque_sem_ce07 = pd.concat([zmb51_pivot_sem_ce07, zstok_pivot_sem_ce07], axis=1)


In [76]:
zmb51_estoque_sem_ce07

,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,2025-04,2025-05,CE05,CE09,CE10,CE11,CE13,media_3m,media_6m
material,,,,,,,,,,,,,,,,,,,
A8K3230,99.0,152.0,62.0,55.0,103.0,66.0,83.0,5.0,74.0,64.0,52.0,56.0,27.0,15.0,NaN,12.0,485.0,248.500000,157.500000
A8K3430,91.0,192.0,65.0,74.0,90.0,121.0,142.0,10.0,156.0,138.0,46.0,96.0,118.0,7.0,1.0,36.0,232.0,89.666667,80.611111
AAV8230,161.0,938.0,436.0,543.0,445.0,522.0,1392.0,388.0,582.0,751.0,565.0,456.0,198.0,77.0,NaN,164.0,1980.0,1072.000000,698.200000
AAV8330,391.0,394.0,419.0,484.0,475.0,454.0,1391.0,350.0,568.0,635.0,482.0,416.0,NaN,68.0,NaN,144.0,2088.0,1116.000000,854.000000
T671600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [77]:
dados = {
    "material": ["A8K3430", "A8K3230", "AAV8230", "AAV8330", "T671600"],
    "possivel_reducao": [11, 4, 50, 30, 2]
}
dados_df = pd.DataFrame(dados).set_index("material")
zmb51_estoque_sem_ce07 = pd.concat([zmb51_pivot_sem_ce07, zstok_pivot_sem_ce07, dados_df], axis=1)


In [78]:
zmb51_estoque_sem_ce07.columns

Index([           2024-06,            2024-07,            2024-08,
                  2024-09,            2024-10,            2024-11,
                  2024-12,            2025-01,            2025-02,
                  2025-03,            2025-04,            2025-05,
                   'CE05',             'CE09',             'CE10',
                   'CE11',             'CE13',         'media_3m',
               'media_6m', 'possivel_reducao'],
      dtype='object')

In [ ]:
zmb51_estoque_sem_ce07_resetado = zmb51_estoque_sem_ce07.reset_index()

zmb51_estoque_sem_ce07_resetado.to_json(
    r"C:\Users\fsp_adolpho.salvador\Desktop\Konica Minolta\Desktop Cloud - Documentos\Desktop\Py\Analise\trasferencias\zmb51_estoque_sem_ce07.json",
    orient="records",
    indent=4
)

In [80]:
zmb51_estoque_sem_ce07_resetado

,material,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12,2025-01,2025-02,...,2025-04,2025-05,CE05,CE09,CE10,CE11,CE13,media_3m,media_6m,possivel_reducao
0,A8K3230,99.0,152.0,62.0,55.0,103.0,66.0,83.0,5.0,74.0,...,52.0,56.0,27.0,15.0,NaN,12.0,485.0,248.500000,157.500000,4
1,A8K3430,91.0,192.0,65.0,74.0,90.0,121.0,142.0,10.0,156.0,...,46.0,96.0,118.0,7.0,1.0,36.0,232.0,89.666667,80.611111,11
2,AAV8230,161.0,938.0,436.0,543.0,445.0,522.0,1392.0,388.0,582.0,...,565.0,456.0,198.0,77.0,NaN,164.0,1980.0,1072.000000,698.200000,50
3,AAV8330,391.0,394.0,419.0,484.0,475.0,454.0,1391.0,350.0,568.0,...,482.0,416.0,NaN,68.0,NaN,144.0,2088.0,1116.000000,854.000000,30
4,T671600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2


In [81]:
fup_pivot = fup.pivot_table(
    index='material',
    columns='data_prev_entrada',
    values='qtde_pedido',
    aggfunc='sum'
)

In [83]:
fup_pivot.columns


DatetimeIndex(['2025-06-10', '2025-06-20', '2025-07-15', '2025-09-09'], dtype='datetime64[ns]', name='data_prev_entrada', freq=None)